In [3]:
%load_ext autoreload
%autoreload 1
%aimport redditClient

In [10]:
from redditClient import redditClient
import matplotlib.pyplot as plt
import matplotlib.dates as md
import praw
import json
from collections import Counter
import time
import praw
from datetime import datetime

In [13]:
# subreddit name we interested in getting the hot submissions
sSubredditName = 'Cricket'
# maximum number of hot submissions
hardLimit = 1000 # Reddit API Limit

In [14]:
# construct Reddit client
# please update the 2auth verification code if necessary
client = redditClient()

In [15]:
# load the subreddit weAreMusicMakers
subreddit = client.subreddit(sSubredditName)
print(subreddit)

Cricket


In [16]:
# This is a test run, to see the content of the retrived data
# and to understand how to store the data for processing
# I have no luck on replace my top posts with latest posts because of an unknown auth error,
# and it doesn't work on rerunning the same original top method either
# as I have to stick with the original, year long top post

allSubmissions = []
# There is no meaningful topic for the latest post, I will do the top post instead
# for submission in subreddit.new(limit=hardLimit):
#     allSubmissions.append(submission)

for submission in subreddit.search("test", time_filter="year", limit=hardLimit):
    allSubmissions.append(submission)
    
len(allSubmissions)

218

In [18]:
# this function is used for convert a list of submission into a json format
# so that I can clean my data without destroy the original version and minimize the API access
#
# In rare situations, I found some replies are replaced with "MoreComments"
# however, most of the time, they are invisible in the offical reddit post, even clicked the show button
# thus a removal for those data
commentIndex = [0]
def convertToJson(submissions:list[praw.models.reddit.submission.Submission])-> ([], int):
    submissionJsonObjList = []
    noTotalComment = 0
    pauseDuration = 7
    index = 0
    
    # iterate each submission and to extract each contents:
    for submission in submissions:
        title = submission.title
        
        print(f"TITLE {index}: {title}")

        content = {}
        content["post_title"]   = submission.title
        content["post_user_id"]  = f"{submission.author}"
        content["post_id"] = submission.id    
        content["post_content"]    = submission.selftext
        content["post_vote"]   = submission.score
        content["created_date"] = submission.created_utc
        content["comment_user_id"] = None
        content["comment"] = None
        content["comment_vote"] = None
        content["comment_id"] = None
   
        # I will unflatten the tree after I have parse them into json
        print(f"Number of Reply: {submission.num_comments}")
        
        while True:
            try:
                repliesTempMain = []
                commentIndex[0] = 0
                repliesTempMain = parseReplies(submission.comments, title)
            except Exception as ex:
                continue
                print("Time out (Replies), Retry...")
                sleep(20)
            break
            
        content["replies"] = repliesTempMain
        submissionJsonObjList.append(content)
        noTotalComment += submission.num_comments
        
        if index % pauseDuration == pauseDuration - 1:
            print(f">> POST DELAY: {index}")
            time.sleep(10)
        
        index += 1
        
    return submissionJsonObjList, noTotalComment
        
    
# This is where the analysis got tricky because it requires recursion
# for the analysis part, I also need to do the same treatment because it is curcial to
# keep the orientation of the replies; otherwise, the replies loses its meanings due to lack of context.

def parseReplies(replies:praw.models.comment_forest.CommentForest, title):
    commentDelay = 1000

    if len(replies) == 0:
        return []
    
    replyList = []
    # generate all replys under the tree
    for reply in replies: 
        
        commentIndex[0] += 1
        
        if isinstance(reply, praw.models.MoreComments):
            # occasionally, instead of comments, it extracts
            # a "moreComments" object which is different from replies.
            # However, after some investigation, because seems like the "moreComments"
            # in the offical reddit page are invisible at the most of the time,
            # while this kind of commenents are exceptionally rare,
            # I decided to skip these replies and focus on the more meaningful one.
            if reply.count > 0:
                if commentIndex[0]>= 1000:
                    print(f"    COMMENT cap reach: {commentIndex}")
                    return []
                while True:
                    replyListTemp = []
                    try:
                        replyListTemp =  parseReplies(reply.comments(), title)
                        break
                    except Exception as ex:
                        print("Time out (more comments), Retry...")
                        time.sleep(20)
                        continue     
                        
                replyList += replyListTemp
        else:
            if commentIndex[0]>= 1000:
                print(f"    COMMENT cap reach: {commentIndex}")
                return []
            
            replyObj = {}
            replyObj["post_title"]   = title
            replyObj["post_user_id"] = None
            replyObj["post_id"]      = None   
            replyObj["post_content"] = None
            replyObj["post_vote"]    = None
            replyObj["created_date"] = None
            replyObj["comment_user_id"] = f"{reply.author}"
            replyObj["comment_time"] = reply.created_utc
            replyObj["comment"]      = reply.body
            replyObj["comment_vote"] = reply.score
            replyObj["comment_id"]   = reply.id
            
            while True:
                repliesTreeTemp = []
                try:
                    repliesTreeTemp      = parseReplies(reply.replies, title)   
                    break
                except Exception as ex:
                    print("Time out (subtree), Retry...")
                    time.sleep(20)
                    continue                 
            
            replyObj["replies"] = repliesTreeTemp
            replyList.append(replyObj)
            
        if commentIndex[0]>= 1000:
            print(f"    COMMENT cap reach: {commentIndex}")
            break
        
    return replyList


start = time.time()
redditJsonObj, noTotalComment = convertToJson(allSubmissions)

# Writing those files into a json file just in case my account got rate limited on accessing the data
with open( "HotRedditPostRelatedToCricketTestGame1000.json" , "w" ) as write:
    json.dump({"Reddit":redditJsonObj} , write, indent=4 )
end = time.time()

print(f"Time taken: {end - start}")
print(f"Total Posts: {len(allSubmissions)}")
print(f"Total Comments: {noTotalComment}")


TITLE 0: Bangladesh beats Pakistan for the first ever in Test Cricket 
Number of Reply: 293
TITLE 1: James Anderson's Final Test Tally against all countries 
Number of Reply: 240
TITLE 2: Match Thread: 1st Test - India vs Bangladesh, Day 1
Number of Reply: 7588
    COMMENT cap reach: [1000]
    COMMENT cap reach: [1000]
    COMMENT cap reach: [1000]
TITLE 3: WTC table after India defeats Bangladesh in the 1st Test
Number of Reply: 222
TITLE 4: Match Thread: Bangladesh v India - 2nd Test, Day 5
Number of Reply: 5058
    COMMENT cap reach: [1000]
TITLE 5: James Anderson announces he will retire from international cricket after England's first Test of the summer against West Indies at Lord's
Number of Reply: 316
TITLE 6: Which Test era would come out on top in a match? 
Number of Reply: 634
>> POST DELAY: 6
TITLE 7: Match Thread: Bangladesh v India - 2nd Test, Day 4
Number of Reply: 8062
    COMMENT cap reach: [1000]
    COMMENT cap reach: [1000]
TITLE 8: Match Thread: 1st Test - India vs

    COMMENT cap reach: [1000]
    COMMENT cap reach: [1000]
    COMMENT cap reach: [1000]
TITLE 65: Match Thread: 3rd Test - England vs Sri Lanka, Day 3
Number of Reply: 2531
    COMMENT cap reach: [1000]
    COMMENT cap reach: [1000]
TITLE 66: Match Thread: 2nd Test - Pakistan vs Bangladesh, Day 4
Number of Reply: 1979
    COMMENT cap reach: [1000]
    COMMENT cap reach: [1000]
TITLE 67: Match Thread: 1st Test - England vs Sri Lanka, Day 1
Number of Reply: 2105
    COMMENT cap reach: [1000]
TITLE 68: 0 Test Match won by Pakistan since March 2022. Is this the worst downfall for Pakistan Test Cricket?
Number of Reply: 123
TITLE 69: Match Thread: 2nd Test - India vs England, Day 1
Number of Reply: 9163
    COMMENT cap reach: [1000]
>> POST DELAY: 69
TITLE 70: Match Thread: 2nd Test - England vs Sri Lanka, Day 1
Number of Reply: 2519
    COMMENT cap reach: [1000]
    COMMENT cap reach: [1000]
TITLE 71: Match Thread: 3rd Test - England vs Sri Lanka, Day 1
Number of Reply: 1957
    COMMENT 

    COMMENT cap reach: [1000]
    COMMENT cap reach: [1000]
    COMMENT cap reach: [1000]
TITLE 120: Ashwin joins Warne in having the second-most 5-fers in Tests
Number of Reply: 86
TITLE 121: Match Thread: 1st Test - Sri Lanka vs New Zealand, Day 2
Number of Reply: 800
TITLE 122: Match Thread: 2nd Test - England vs Sri Lanka, Day 4
Number of Reply: 1167
    COMMENT cap reach: [1000]
    COMMENT cap reach: [1000]
TITLE 123: Match Thread: 2nd Test - Australia vs West Indies, Day 1
Number of Reply: 3319
    COMMENT cap reach: [1000]
    COMMENT cap reach: [1000]
    COMMENT cap reach: [1000]
TITLE 124: Match Thread: 2nd Test - New Zealand vs Australia, Day 1
Number of Reply: 4625
    COMMENT cap reach: [1000]
    COMMENT cap reach: [1000]
    COMMENT cap reach: [1000]
TITLE 125: Match Thread: 1st Test - Australia vs New Zealand, Day 3
Number of Reply: 6309
Time out (more comments), Retry...
    COMMENT cap reach: [1000]
>> POST DELAY: 125
TITLE 126: England announce their XI for the firs

TITLE 187: Post Day Thread: India v England, 1st Test, Day 2
Number of Reply: 661
TITLE 188: Most 5 wicket hauls in Test Cricket.
Number of Reply: 159
>> POST DELAY: 188
TITLE 189: Match Thread: 1st Test - Australia vs New Zealand, Day 4
Number of Reply: 1870
    COMMENT cap reach: [1000]
TITLE 190: WTC table after Bangladesh defeats Pakistan in the 2nd Test
Number of Reply: 122
TITLE 191: 104 from Kane Williamson, the first test century at Sylhet and his 29th
Number of Reply: 114
TITLE 192: Match Thread: 1st Test - South Africa vs West Indies, Day 5
Number of Reply: 656
TITLE 193: Historic day for India as star quick reaches rankings pinnacle (Bumrah becomes No. 1 Test Bowler)
Number of Reply: 192
TITLE 194: WTC table after India defeats England in the 5th Test
Number of Reply: 208
TITLE 195: Match Thread: 2nd Test - South Africa vs New Zealand, Day 4
Number of Reply: 2254
    COMMENT cap reach: [1000]
    COMMENT cap reach: [1000]
    COMMENT cap reach: [1000]
>> POST DELAY: 195
TITL

In [19]:
# I commented these lines because I hardly found any meaningful during the analysis
# with open('RedditPostAndCommentsRaw.json') as f:
# with open('LatestRedditPostsAndCommentsRaw.json') as f:
with open('HotRedditPostRelatedToCricketTestGame1000.json') as f:
    dictson = json.load(f)
#     print(d)

submissions:[] = dictson.get("Reddit")

In [20]:
# In the procedure, I am going to flatten the comment becase the main idea of the analysis is to understand how users
# response with certain type of posts and the overall response attutude, so the hierarchy of the replies are ignored.
# After the procedure, I will have a list submittions contains a list of comment instead of a tree.
# Also, this process will make data cleaning and training much easier.
#
# However, submission is remain untouch because I might do analysis separately for higher and lower vote counts

def flattenTree(replies:[], flattenComments: []):
    if len(replies) == 0:
        return []
    for reply in replies:
        replyChainFreeReply = reply.copy()
        replyChainFreeReply.pop("replies")
        flattenComments.append(replyChainFreeReply)
        
        flattenTree(reply.get("replies"), flattenComments)
        
    return flattenComments
        
    
def flattenCommentTree(submission:dict):
    
    flattenSubmission = submission.copy()
    flattenSubmission["replies"] = flattenTree(submission.get("replies"), [])
    
    return flattenSubmission

submissionsLen = len(submissions)

flattenSubmissions = []
for submission in submissions:
    flattenSubmission = flattenCommentTree(submission)
    
    flattenSubmissions.append(flattenSubmission)
    

# Flattened result:
flattenSubmissions[0]

{'post_title': 'Bangladesh beats Pakistan for the first ever in Test Cricket ',
 'post_user_id': 'NoQuestion4045',
 'post_id': '1f0syen',
 'post_content': '',
 'post_vote': 2445,
 'created_date': 1724580261.0,
 'comment_user_id': None,
 'comment': None,
 'comment_vote': None,
 'comment_id': None,
 'replies': [{'post_title': 'Bangladesh beats Pakistan for the first ever in Test Cricket ',
   'post_user_id': None,
   'post_id': None,
   'post_content': None,
   'post_vote': None,
   'created_date': None,
   'comment_user_id': 'None',
   'comment_time': 1724580448.0,
   'comment': 'Fort Rawalpindi, yet to be captured, by the home side 😔',
   'comment_vote': 1151,
   'comment_id': 'lju4ld6'},
  {'post_title': 'Bangladesh beats Pakistan for the first ever in Test Cricket ',
   'post_user_id': None,
   'post_id': None,
   'post_content': None,
   'post_vote': None,
   'created_date': None,
   'comment_user_id': 'mofucker20',
   'comment_time': 1724583533.0,
   'comment': 'That stat is so rid

### Pour into a CSV file

In [22]:
import csv
with open('HotRedditPostRelatedToCricketTestGame1000.csv', 'w', newline='', encoding="utf-8") as csvfile:
    
    fieldnames = ['post_title', 'post_user_id', 'post_id', 'post_content', 'post_vote', 'created_date', 'comment_user_id', 'comment', 'comment_vote', 'comment_id', 'comment_time']
    writer = csv.writer(csvfile)
    writer.writerow(fieldnames)
    
    redditCsvWriter = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    #     redditCsvWriter.writerow(['Spam'] * 5 + ['Baked Beans'])
    for submissions in flattenSubmissions:
        entries = {}
        entries['post_title']      = submissions.get('post_title')
        entries['post_user_id']    = submissions.get('post_user_id')
        entries['post_id']         = submissions.get('post_id')
        entries['post_content']    = submissions.get('post_content')
        entries['post_vote']       = submissions.get('post_vote')
        entries['created_date']    = submissions.get('created_date')
        entries['comment_user_id'] = submissions.get('comment_user_id')
        entries['comment_user_id'] = submissions.get('comment_user_id')
        entries['comment']         = submissions.get('comment')
        entries['comment_vote']    = submissions.get('comment_vote')
        entries['comment_id']      = submissions.get('comment_id')
        entries["comment_time"]    = submissions.get('comment_time')
        
        redditCsvWriter.writerow(entries)
        
        for reply in submissions.get('replies'):
            replyEntries = {}
            replyEntries['post_title']      = reply.get('post_title')
            replyEntries['post_user_id']    = reply.get('post_user_id')
            replyEntries['post_id']         = reply.get('post_id')
            replyEntries['post_content']    = reply.get('post_content')
            replyEntries['post_vote']       = reply.get('post_vote')
            replyEntries['created_date']    = reply.get('created_date')
            replyEntries['comment_user_id'] = reply.get('comment_user_id')
            replyEntries['comment']         = reply.get('comment')
            replyEntries['comment_vote']    = reply.get('comment_vote')
            replyEntries['comment_id']      = reply.get('comment_id')  
            replyEntries["comment_time"]    = reply.get('comment_time')
            
            redditCsvWriter.writerow(replyEntries)

    
    